In [1]:
%load_ext dotenv
%dotenv

In [14]:
from langchain.prompts import PromptTemplate
from datetime import datetime

In [10]:
# prompt with no input
no_input_prompt = PromptTemplate(input_variables=[], template='give me a class name in wow')
no_input_prompt.format()

'give me a class name in wow'

In [11]:
# prompt with 1 input
one_input_prompt = PromptTemplate(
    input_variables=['game_class'], template='give me a {game_class} name in wow')
one_input_prompt.format(game_class='heal class')

'give me a heal class name in wow'

In [12]:
# prompt with 2 inputs
two_input_prompt = PromptTemplate(
    input_variables=['game_class', 'game_name'], template='give me a {game_class} name in {game_name}')
two_input_prompt.format(game_class='heal class', game_name='ff14')

'give me a heal class name in ff14'

In [13]:
# another way to get tempalte
template = 'give me a {game_class} name in {game_name}'
prompt_template = PromptTemplate.from_template(template)
prompt_template.input_variables

['game_class', 'game_name']

In [16]:
# partial inputs
prompt = PromptTemplate(
    template="tell me the average temperature of {city} at {date} of {year}",
    input_variables=['city', 'date', 'year']
)
partial_prompt = prompt.partial(city='LA')

partial_prompt.format(year=2000, date='Dec 12')

'tell me the average temperature of LA at Dec 12 of 2000'

In [17]:
#you can also use function in the partial prompt
def _get_datetime():
    now=datetime.now()
    return now.strftime('%m-%d')

prompt = PromptTemplate(
    template="tell me the average temperature of {city} at {date} of {year}",
    input_variables=['city', 'date', 'year']
)
partial_prompt = prompt.partial(city='LA', date=_get_datetime)
partial_prompt.format(year=2000)

'tell me the average temperature of LA at 02-15 of 2000'

### few shot: 
- providing a few examples to the model let the model know how to handle certain question (not changing the model itself)
- fine tuning: directly modify the model itself with a lot more examples (will change the model itself)

In [20]:
from langchain.prompts.few_shot import FewShotPromptTemplate

examples = [
    {
        "question": "how are you doing?",
        "answer": "Greetings! Im doing well"
    },
    {
        "question": "what's the date today?",
        "answer": "Greetings! Today is Thursday"
    },
    {
        "question": "how is the weather now?",
        "answer": "Greetings! The weather is nice"
    }
]

# apply the prompt template to the examples above
example_prompt = PromptTemplate(
    input_variables=['question', 'answer'],
    template="Question: {question}\n{answer}"
    )

prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=['input']
)

print(prompt.format(input="what to learn today?"))

Question: how are you doing?
Greetings! Im doing well

Question: what's the date today?
Greetings! Today is Thursday

Question: how is the weather now?
Greetings! The weather is nice

Question: what to learn today?


In [21]:
from langchain.llms import OpenAI

In [22]:
llm = OpenAI()

In [23]:
llm.predict('Question: what to learn today?')

"\n\n1. A new language: Learning a new language can open up opportunities for travel, career advancement, and personal enrichment.\n\n2. Coding/programming: In today's digital age, having coding skills can be highly beneficial for job prospects and understanding technology.\n\n3. Photography: Learning how to take great photos can help you capture memories and express your creativity.\n\n4. Cooking/baking: Learning new recipes and cooking techniques can improve your health, save you money, and impress your friends and family.\n\n5. Personal finance: Understanding how to manage your money, invest, and save for the future is a valuable life skill.\n\n6. Meditation/mindfulness: Practicing mindfulness and meditation can reduce stress, improve focus, and promote overall well-being.\n\n7. First aid/CPR: Knowing basic first aid and CPR can potentially save a life in an emergency situation.\n\n8. Graphic design: Learning graphic design skills can help you create visually appealing content for p

In [24]:
# as you can see, the key difference is the "greetings" at the beginning of the response
llm.predict(prompt.format(input="what to learn today?"))

'\nGreetings! That depends on your interests and goals. What subjects or skills are you interested in learning?'

ExampleSelector
- when you have a lot of examples, you could pass selected examples to few shot based on certain criteria (such as similarity)
- it can also reduce the number of tokens sent to LLM, which could help reduce cost
- Example: SemanticSimilarityExampleSelector

In [26]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Examples of a pretend task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [30]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # The number of examples to produce.
    k=1,
)
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

In [31]:
# Input is a feeling, so should select the happy/sad example
print(similar_prompt.format(adjective="worried"))

Give the antonym of every input

Input: happy
Output: sad

Input: worried
Output:


In [32]:
# Input is a measurement, so should select the tall/short example
print(similar_prompt.format(adjective="large"))

Give the antonym of every input

Input: tall
Output: short

Input: large
Output:


In [33]:
# You can add new examples to the SemanticSimilarityExampleSelector as well
similar_prompt.example_selector.add_example(
    {"input": "enthusiastic", "output": "apathetic"}
)
print(similar_prompt.format(adjective="passionate"))

Give the antonym of every input

Input: enthusiastic
Output: apathetic

Input: passionate
Output:


### llm module
- predict(): direct call for 1 input
- generate(): batch processing

In [34]:
llm.predict('tell me a joke')

'\n\nWhy was the math book sad?\n\nBecause it had too many problems.'

In [35]:
res = llm.generate(['tell me a joke', 'whos the best player in NBA'])

In [36]:
res

LLMResult(generations=[[Generation(text="\n\nWhy couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!", generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nThe best player in the NBA is currently debatable, as it is subjective and can vary based on individual opinions. Some people may argue that LeBron James, Kevin Durant, or Kawhi Leonard are the best players, while others may argue for players like Giannis Antetokounmpo, Stephen Curry, or James Harden. Ultimately, it is up to personal preference and can change over time.', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'total_tokens': 110, 'prompt_tokens': 11, 'completion_tokens': 99}, 'model_name': 'gpt-3.5-turbo-instruct'}, run=[RunInfo(run_id=UUID('4cb0341b-4de7-4792-a6c1-3e81e136c47a')), RunInfo(run_id=UUID('e8959017-c3e7-45e9-9b9b-467b1a8d991d'))])

In [39]:
res.generations[0][0].text

"\n\nWhy couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!"

In [40]:
res.generations[1][0].text

'\n\nThe best player in the NBA is currently debatable, as it is subjective and can vary based on individual opinions. Some people may argue that LeBron James, Kevin Durant, or Kawhi Leonard are the best players, while others may argue for players like Giannis Antetokounmpo, Stephen Curry, or James Harden. Ultimately, it is up to personal preference and can change over time.'

### customize LLM module
- to wrap the new models that are not supported in langchain 
- to create similation tests
- to define when certain inputs are provided, how the output would be

In [41]:
from typing import Any, List, Mapping, Optional
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun

In [50]:
class SilentAI(LLM):

    @property
    def _llm_type(self) -> str:
        return "SilentAI"
    
    def _call(
            self,
            prompt: str,
            stop: Optional[List[str]] = None,
            run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted")
        pd = prompt.find("?")
        if pd >= 0:
            return prompt[0:pd] + " o7"
        return "0.0"

In [51]:
cllm = SilentAI()
print(cllm)

SilentAI
Params: {}


In [52]:
cllm.predict("how are you?")

'how are you o7'

In [53]:
cllm.predict("how is food?")

'how is food o7'

In [54]:
cllm.predict("i dont like your response")

'0.0'